In [ ]:
# Henry Judkins
# Using the tensorflow dataset/model to identify different kinds of flowers



import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt

# Downloading the flower photos from tensorflow 
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)

# Finding the directory with the flower photos
data_dir = pathlib.Path(archive)

# Updating data_dir to point to the correct sub-directory
data_dir = data_dir / 'flower_photos'

# Defining parameters for the loader
batch_size = 32
img_height = 180
img_width = 180

# Creating the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Creating the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

normalization_layer = tf.keras.layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)


(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

class_names = train_ds.class_names
printf(f"Class names: {class_names}")

# image_path = 'path'


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Epoch 1/3


2025-09-03 17:40:12.566568: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 129777664 exceeds 10% of free system memory.
2025-09-03 17:40:13.722633: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 32444416 exceeds 10% of free system memory.
2025-09-03 17:40:13.760724: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 31002624 exceeds 10% of free system memory.
2025-09-03 17:40:14.386113: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 29047680 exceeds 10% of free system memory.
2025-09-03 17:40:14.515656: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 31002624 exceeds 10% of free system memory.


92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 880ms/step - accuracy: 0.4390 - loss: 1.3091 - val_accuracy: 0.5913 - val_loss: 1.0379
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 655ms/step - accuracy: 0.6073 - loss: 0.9900 - val_accuracy: 0.6253 - val_loss: 0.9885
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 60s 648ms/step - accuracy: 0.6819 - loss: 0.8440 - val_accuracy: 0.6649 - val_loss: 0.8859


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  2.52s/ url]


Dataset tf_flowers downloaded and prepared to /home/codespace/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
